# imports:

In [1]:
from os import listdir,mkdir,makedirs
from os.path import isfile, join, isdir,exists
import pandas as pd
import numpy as np
from scipy import stats
import re
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from myplots import roundup, rounddown, find_decimal_fold, percentile_cut_off, rarefaction_calc, rarefaction_plot,draw_correlation_scatter
from matplotlib.ticker import FormatStrFormatter
import cPickle as pickle
from Bio.SeqUtils import GC
import seaborn as sns
import random
from scipy.stats import pearsonr,fisher_exact
from skbio.diversity.alpha import shannon, simpson, berger_parker_d

from pop_organize import get_sample_data, get_sample_with_dfs
from SufficientStatistics.SufficientStatistics import *
from MyFunctionsShani import *
import math
from myplots import roundup, rounddown, find_decimal_fold
from skbio.stats.distance import mantel
from scipy.spatial.distance import braycurtis, pdist


from GeneralFeaturePhenotypeInteractions.Feature_phenotype_functions import * 
from TCR_microbiome_interactions.TCR_microbiome_interactions_functions import *
from TCR_microbiome_interactions.TCR_microbiome_interactions_functions2 import *
from SampleLists.SampleFileFunctions import *
from PhenotypicData.PhenotypeGenerationFunctions import *
from CardioProject.CardioFunctions import *
from PredictionPipeline.PredictionFunctions import *

import os
from Utils import cacheOnDisk
from SegalQueue.qp import qp,fakeqp
from addloglevels import sethandlers

#ML imports:
from xgboost import XGBClassifier
import lightgbm as lgb
from collections import OrderedDict
from sklearn.model_selection import GroupKFold,StratifiedKFold, KFold
import statsmodels.formula.api as sm
from sklearn.linear_model import LogisticRegression
import shap
from sklearn import metrics, preprocessing
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel,SelectKBest,chi2,mutual_info_classif,f_classif
from sklearn.naive_bayes import GaussianNB

MyPath='/net/mraid08/export/genie/Lab/Personal/ShaniBAF'

done1
stop
stop
done1
stop
done
done
done
done
stop
stop
end of function!!!


/usr/wisdom/python/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import time
cdate=str(time.strftime("%d%m%Y"))
cdate

'25082018'

# predicting isCardio using all Samples

## prepare phenotypeDF, TCRdf, TCRfeatureDF

### phenotype DF

### TCRdf

In [ ]:

# parameters to change:
#general parameters:
DFtype='TCR'
genDF=False #False=generate new df
toBinary=True

#basic prameters for generating df:
mbLevel='g'
useShortName=True

datasetFolder='%s/TCR_real_data/PNP530Cardio126Combined' %MyPath
datasetName='PNP530Cardio126' 

#parameters for manipulations on df 
minVal=None    #minVal can be None,0, float, or 'dfMinVal' or dfMinVal2:
minSharedT=None #minimal number of samples shared by seq/species required to leave sample in the database (int or None)
percShared=10 #minimal number of samples shared by seq/species required to leave sample in the database (int [ percentage]
                #or None)
removeOutliers=True
normData=True
logTransform=True




#constant parameters:
extractUniqueAA=False # use True when this is the first time to analyze this dataset, otherwise, use False
minNshared=5
onlyProductive=True

mbDataFolder='AllSeqProjects'
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/PNP530','rb') as fp:
    PNP530=pickle.load(fp)
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/Cardio126','rb') as fp:
    Cardio126=pickle.load(fp)
PNP530Cardio126list=PNP530+Cardio126
SampleList=PNP530Cardio126list
SampleListName='PNP530Cardio126list'
libPrepMethod=None
filterGenotek=True
filterMinimalReads=9000000
filterlibPrepMethod=libPrepMethod
groupFunction='noOutlierMean'
nSTD=5
nMinSamples=3
ignoreNotSameYear=True
removeSamePerson=False




df=genTCRorMBdfWithManipulations(DFtype,genDF,toBinary,removeOutliers,normData,logTransform,
                                 minVal,minSharedT,percShared,
                                 mbLevel,useShortName,datasetFolder,datasetName,extractUniqueAA,
                                minNshared,onlyProductive,mbDataFolder,SampleList,
                                  SampleListName,filterlibPrepMethod,filterGenotek, 
                                  groupFunction, nSTD, nMinSamples, ignoreNotSameYear, removeSamePerson)


## run model:

### phenotypes only, try different models:

In [ ]:
modelAndParams_list=[(LogisticRegression,'LogisticRegression',{}),
                   (lgb.LGBMClassifier,'LGBMClassifier',{'learning_rate': 0.1, 'n_estimators': 1000,'max_depth': 3,'num_threads':2}),
                   (XGBClassifier,'XGBClassifier',{'learning_rate': 0.05, 'n_estimators': 100,'max_depth': 6,'num_threads':2}),
                   (GaussianNB,'GaussianNB',{})]


for n,modelAndParams in enumerate(modelAndParams_list):
    
    model=modelAndParams[0]
    modelName=modelAndParams[1]
    model_params=modelAndParams[2]
    
    
    print n,modelName
    
    # (1) general definitions:
    datasetType='PNP_Cardio' #PNP/PNP_Cardio
    #nSeqSS=9000 #5000/9000/15000/'all'

    #(2) model definitions:
    ResultFolder='%s/TCR_real_data/Predictions' %MyPath
    if not isdir(ResultFolder):
        makedirs(ResultFolder)

    n_splits=3
    useCV=True
    stratifiedCV=True

    #(3) target definition:
    targetDF=pd.DataFrame(index=['BD'+str(x) for x in range(1,1100)])
    targetDF['isCardio']=np.where(targetDF.index.str.replace('BD','').astype(int)>949,1,0)
    targetName='isCardio'
    YName=targetName

    #  (4) feature composition:
    XName='PNP530Cardio126all_phenotypesOnly'
    ## naming instructions: phenotypes: indicate old or new, TCRdf-indicate threshold to inclusion and binary/RA, 
    ## PCA- indicate n_comp and is Sparse, indicate feature selection
    usePhenotype=True #True/False
    useTCRdf=False #True/False
    useTCRfeatures=False #True/False
    usePCAdf=False #True/False

    # (5) phenotypeDF:

    genPhenotypeDF=True #True/False. no sense for 'True'  if usePhenotype=False
    phenotypefile=None
    phenotypeDFname='newSmall' #('small','allNum and more- see step #6.2


    # (6) TCRdf:
    datasetFolder='%s/TCR_real_data/PNP530Cardio126Combined' %MyPath
    datasetName='PNP530Cardio126' 
    TCRdfName='sharingMatrix_PNP530Cardio126_minNshared5_RA_onlyProductiveTrue__percShared10_OLtrimmed_binary'

    #(7) PCAdf (based on the TCRdf selected)
    n_comp=100 # int or None 
    isSparse=True #True/False

    #(8) TCRfeatures:
    getTCRfeatures=False #True/False. do not use True when useTCRfeatures=False

    #(9) X and y processing:
#     folderToSampleList='%s/SamplesForAnalysis_corrected' %datasetFolder
#     sampleListName='matched86List' ################IMPORTANT!!! ###################
#     matched86List=gen_sampleList_from_Folder(folderToSampleList,'PNP530Cardio126Matched86')
    with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/PNP530','rb') as fp:
        PNP530=pickle.load(fp)
    with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/Cardio126','rb') as fp:
        Cardio126=pickle.load(fp)
    PNP530Cardio126list=PNP530+Cardio126
    sampleList=PNP530Cardio126list
    sampleListName='PNP530Cardio126list'

    filterFeaturesByCorr=None #None or float between 0 and 1
    featureSelectionUsingModel=False # True/False
    C=0.5 #None (if none, C will be set to 0.1) or float between 0 and 1
    filterFeaturesByUnivariate=f_classif     # None/chi2/f_classif/mutual_info_classif
    k=150 #None (will be set to 50 / int - defines number of k best))


    predictionPipeline(datasetType,ResultFolder,model,modelName,model_params,n_splits, useCV,stratifiedCV,
                          XName,usePhenotype,useTCRdf,useTCRfeatures,usePCAdf,
                          YName,targetDF,
                          genPhenotypeDF,phenotypefile,phenotypeDFname,
                          datasetFolder,datasetName,TCRdfName,n_comp,isSparse,
                          getTCRfeatures,sampleListName,sampleList,
                          filterFeaturesByCorr,featureSelectionUsingModel,C,
                          filterFeaturesByUnivariate,k)

    

In [ ]:
print 'dfs'

### phenotypes+PCA+TCRdf, try different models

In [ ]:
modelAndParams_list=[(LogisticRegression,'LogisticRegression',{}),
                   (lgb.LGBMClassifier,'LGBMClassifier',{'learning_rate': 0.1, 'n_estimators': 1000,'max_depth': 3,'num_threads':2}),
                   (XGBClassifier,'XGBClassifier',{'learning_rate': 0.05, 'n_estimators': 100,'max_depth': 6,'num_threads':2}),
                   (GaussianNB,'GaussianNB',{})]


for n,modelAndParams in enumerate(modelAndParams_list):
    
    model=modelAndParams[0]
    modelName=modelAndParams[1]
    model_params=modelAndParams[2]
    
    
    print n,modelName
    
    # (1) general definitions:
    datasetType='PNP_Cardio' #PNP/PNP_Cardio
    #nSeqSS=9000 #5000/9000/15000/'all'

    #(2) model definitions:
    ResultFolder='%s/TCR_real_data/Predictions' %MyPath
    if not isdir(ResultFolder):
        makedirs(ResultFolder)

    n_splits=3
    useCV=True
    stratifiedCV=True

    #(3) target definition:
    targetDF=pd.DataFrame(index=['BD'+str(x) for x in range(1,1100)])
    targetDF['isCardio']=np.where(targetDF.index.str.replace('BD','').astype(int)>949,1,0)
    targetName='isCardio'
    YName=targetName

    #  (4) feature composition:
    XName='PNP530Cardio126all_phenotypes&PCA&TCRfeatures'
    ## naming instructions: phenotypes: indicate old or new, TCRdf-indicate threshold to inclusion and binary/RA, 
    ## PCA- indicate n_comp and is Sparse, indicate feature selection
    usePhenotype=True #True/False
    useTCRdf=False #True/False
    useTCRfeatures=True #True/False
    usePCAdf=True #True/False

    # (5) phenotypeDF:

    genPhenotypeDF=True #True/False. no sense for 'True'  if usePhenotype=False
    phenotypefile=None
    phenotypeDFname='newSmall' #('small','allNum and more- see step #6.2


    # (6) TCRdf:
    datasetFolder='%s/TCR_real_data/PNP530Cardio126Combined' %MyPath
    datasetName='PNP530Cardio126' 
    TCRdfName='sharingMatrix_PNP530Cardio126_minNshared5_RA_onlyProductiveTrue__percShared10_OLtrimmed_binary'

    #(7) PCAdf (based on the TCRdf selected)
    n_comp=10 # int or None 
    isSparse=True #True/False

    #(8) TCRfeatures:
    getTCRfeatures=False #True/False. do not use True when useTCRfeatures=False

    #(9) X and y processing:
#     folderToSampleList='%s/SamplesForAnalysis_corrected' %datasetFolder
#     sampleListName='matched86List' ################IMPORTANT!!! ###################
#     matched86List=gen_sampleList_from_Folder(folderToSampleList,'PNP530Cardio126Matched86')
    with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/PNP530','rb') as fp:
        PNP530=pickle.load(fp)
    with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/Cardio126','rb') as fp:
        Cardio126=pickle.load(fp)
    PNP530Cardio126list=PNP530+Cardio126
    sampleList=PNP530Cardio126list
    sampleListName='PNP530Cardio126list'

    filterFeaturesByCorr=None #None or float between 0 and 1
    featureSelectionUsingModel=False # True/False
    C=0.5 #None (if none, C will be set to 0.1) or float between 0 and 1
    filterFeaturesByUnivariate=f_classif     # None/chi2/f_classif/mutual_info_classif
    k=150 #None (will be set to 50 / int - defines number of k best))


    predictionPipeline(datasetType,ResultFolder,model,modelName,model_params,n_splits, useCV,stratifiedCV,
                          XName,usePhenotype,useTCRdf,useTCRfeatures,usePCAdf,
                          YName,targetDF,
                          genPhenotypeDF,phenotypefile,phenotypeDFname,
                          datasetFolder,datasetName,TCRdfName,n_comp,isSparse,
                          getTCRfeatures,sampleListName,sampleList,
                          filterFeaturesByCorr,featureSelectionUsingModel,C,
                          filterFeaturesByUnivariate,k)

    

### phenotypes+PCA, try different models:

In [ ]:
modelAndParams_list=[(LogisticRegression,'LogisticRegression',{}),
                   (lgb.LGBMClassifier,'LGBMClassifier',{'learning_rate': 0.1, 'n_estimators': 1000,'max_depth': 3,'num_threads':2}),
                   (XGBClassifier,'XGBClassifier',{'learning_rate': 0.05, 'n_estimators': 100,'max_depth': 6,'num_threads':2}),
                   (GaussianNB,'GaussianNB',{})]


for n,modelAndParams in enumerate(modelAndParams_list):
    
    model=modelAndParams[0]
    modelName=modelAndParams[1]
    model_params=modelAndParams[2]
    
    
    print n,modelName
    
    # (1) general definitions:
    datasetType='PNP_Cardio' #PNP/PNP_Cardio
    #nSeqSS=9000 #5000/9000/15000/'all'

    #(2) model definitions:
    ResultFolder='%s/TCR_real_data/Predictions' %MyPath
    if not isdir(ResultFolder):
        makedirs(ResultFolder)

    n_splits=3
    useCV=True
    stratifiedCV=True

    #(3) target definition:
    targetDF=pd.DataFrame(index=['BD'+str(x) for x in range(1,1100)])
    targetDF['isCardio']=np.where(targetDF.index.str.replace('BD','').astype(int)>949,1,0)
    targetName='isCardio'
    YName=targetName

    #  (4) feature composition:
    XName='PNP530Cardio126all_phenotypes&PCA'
    ## naming instructions: phenotypes: indicate old or new, TCRdf-indicate threshold to inclusion and binary/RA, 
    ## PCA- indicate n_comp and is Sparse, indicate feature selection
    usePhenotype=True #True/False
    useTCRdf=False #True/False
    useTCRfeatures=False #True/False
    usePCAdf=True #True/False

    # (5) phenotypeDF:

    genPhenotypeDF=True #True/False. no sense for 'True'  if usePhenotype=False
    phenotypefile=None
    phenotypeDFname='newSmall' #('small','allNum and more- see step #6.2


    # (6) TCRdf:
    datasetFolder='%s/TCR_real_data/PNP530Cardio126Combined' %MyPath
    datasetName='PNP530Cardio126' 
    TCRdfName='sharingMatrix_PNP530Cardio126_minNshared5_RA_onlyProductiveTrue__percShared10_OLtrimmed_binary'

    #(7) PCAdf (based on the TCRdf selected)
    n_comp=10 # int or None 
    isSparse=True #True/False

    #(8) TCRfeatures:
    getTCRfeatures=False #True/False. do not use True when useTCRfeatures=False

    #(9) X and y processing:
#     folderToSampleList='%s/SamplesForAnalysis_corrected' %datasetFolder
#     sampleListName='matched86List' ################IMPORTANT!!! ###################
#     matched86List=gen_sampleList_from_Folder(folderToSampleList,'PNP530Cardio126Matched86')
    with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/PNP530','rb') as fp:
        PNP530=pickle.load(fp)
    with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/Cardio126','rb') as fp:
        Cardio126=pickle.load(fp)
    PNP530Cardio126list=PNP530+Cardio126
    sampleList=PNP530Cardio126list
    sampleListName='PNP530Cardio126list'

    filterFeaturesByCorr=None #None or float between 0 and 1
    featureSelectionUsingModel=False # True/False
    C=0.5 #None (if none, C will be set to 0.1) or float between 0 and 1
    filterFeaturesByUnivariate=f_classif     # None/chi2/f_classif/mutual_info_classif
    k=150 #None (will be set to 50 / int - defines number of k best))


    predictionPipeline(datasetType,ResultFolder,model,modelName,model_params,n_splits, useCV,stratifiedCV,
                          XName,usePhenotype,useTCRdf,useTCRfeatures,usePCAdf,
                          YName,targetDF,
                          genPhenotypeDF,phenotypefile,phenotypeDFname,
                          datasetFolder,datasetName,TCRdfName,n_comp,isSparse,
                          getTCRfeatures,sampleListName,sampleList,
                          filterFeaturesByCorr,featureSelectionUsingModel,C,
                          filterFeaturesByUnivariate,k)

    

#### plot results:

In [ ]:
f1='%s/TCR_real_data/Predictions/isCardio_predictions/modelComparison_170718 - FullPNP530Caardio126.xlsx' %MyPath
resultDF=pd.read_excel(f1)
title='ROC-AUC and PR-AUC (corrected)\nonly-phenotypes vs.\
phenotypes+TCRseqPCA+TCRfeatures\nFull PNP530 and Cardio126 cohorts'

fig=plot_modelComparison(resultDF,title)

### phenotypes+PCA+features, optimize LGBM:

In [ ]:
param_list=[{'learning_rate': 0.1, 'n_estimators': 1000,'max_depth': 3,'num_threads':2},
            {'learning_rate': 0.1, 'n_estimators': 1000,'max_depth': 3,'num_threads':2,
             'bagging_fraction':0.5,'bagging_freq':1, 'bagging_seed':3}]
featureSelectionUsingModel_list=[True,False]
k_list=[50,100,150]
n_splits_list=[3,5,10]

model=lgb.LGBMClassifier
modelName='LGBMClassifier'

for model_params in param_list:
    for featureSelectionUsingModel in featureSelectionUsingModel_list:
        for k in k_list:
            for n_splits in n_splits_list:
    
                # (1) general definitions:
                datasetType='PNP_Cardio' #PNP/PNP_Cardio
                #nSeqSS=9000 #5000/9000/15000/'all'

                #(2) model definitions:
                ResultFolder='%s/TCR_real_data/Predictions' %MyPath
                if not isdir(ResultFolder):
                    makedirs(ResultFolder)
                useCV=True
                stratifiedCV=True

                #(3) target definition:
                targetDF=pd.DataFrame(index=['BD'+str(x) for x in range(1,1100)])
                targetDF['isCardio']=np.where(targetDF.index.str.replace('BD','').astype(int)>949,1,0)
                targetName='isCardio'
                YName=targetName

                #  (4) feature composition:
                XName='PNP530Cardio126all_phenotypes&PCA&TCRfeatures'
                ## naming instructions: phenotypes: indicate old or new, TCRdf-indicate threshold to inclusion and binary/RA, 
                ## PCA- indicate n_comp and is Sparse, indicate feature selection
                usePhenotype=True #True/False
                useTCRdf=False #True/False
                useTCRfeatures=True #True/False
                usePCAdf=True #True/False

                # (5) phenotypeDF:

                genPhenotypeDF=True #True/False. no sense for 'True'  if usePhenotype=False
                phenotypefile=None
                phenotypeDFname='newSmall' #('small','allNum and more- see step #6.2


                # (6) TCRdf:
                datasetFolder='%s/TCR_real_data/PNP530Cardio126Combined' %MyPath
                datasetName='PNP530Cardio126' 
                TCRdfName='sharingMatrix_PNP530Cardio126_minNshared5_RA_onlyProductiveTrue__percShared10_OLtrimmed_binary'

                #(7) PCAdf (based on the TCRdf selected)
                n_comp=100 # int or None 
                isSparse=True #True/False

                #(8) TCRfeatures:
                getTCRfeatures=False #True/False. do not use True when useTCRfeatures=False

                #(9) X and y processing:
            #     folderToSampleList='%s/SamplesForAnalysis_corrected' %datasetFolder
            #     sampleListName='matched86List' ################IMPORTANT!!! ###################
            #     matched86List=gen_sampleList_from_Folder(folderToSampleList,'PNP530Cardio126Matched86')
                with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/PNP530','rb') as fp:
                    PNP530=pickle.load(fp)
                with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/Cardio126','rb') as fp:
                    Cardio126=pickle.load(fp)
                PNP530Cardio126list=PNP530+Cardio126
                sampleList=PNP530Cardio126list
                sampleListName='PNP530Cardio126list'

                filterFeaturesByCorr=None #None or float between 0 and 1
                featureSelectionUsingModel=False # True/False
                C=0.5 #None (if none, C will be set to 0.1) or float between 0 and 1
                filterFeaturesByUnivariate=f_classif     # None/chi2/f_classif/mutual_info_classif

                predictionPipeline(datasetType,ResultFolder,model,modelName,model_params,n_splits, useCV,stratifiedCV,
                                      XName,usePhenotype,useTCRdf,useTCRfeatures,usePCAdf,
                                      YName,targetDF,
                                      genPhenotypeDF,phenotypefile,phenotypeDFname,
                                      datasetFolder,datasetName,TCRdfName,n_comp,isSparse,
                                      getTCRfeatures,sampleListName,sampleList,
                                      filterFeaturesByCorr,featureSelectionUsingModel,C,
                                      filterFeaturesByUnivariate,k)



#### get results:

In [ ]:
folder='%s/TCR_real_data/Predictions/isCardio_predictions/%s/predictionDFs' %(MyPath,modelName)
results=concat_summarizing_dfs_excel(folder)
results=results.sort_values(by='roc_auc', ascending=False)
f1='%s/TCR_real_data/Predictions/isCardio_predictions/%s/resultSummary_%s.xlsx' %(MyPath, modelName,cdate)
results.to_excel(f1)
results.head(20)

### phenotypes+PCA+features, optimize XGB:

In [ ]:
param_list=[{'learning_rate': 0.05, 'n_estimators': 100,'max_depth': 6,'num_threads':2},
            {'learning_rate': 0.05, 'n_estimators': 100,'max_depth': 6,'num_threads':2,
             'bagging_fraction':0.5,'bagging_freq':1, 'bagging_seed':3}]
featureSelectionUsingModel_list=[True,False]
k_list=[50,100,150]
n_splits_list=[3,5,10]
n_comp_list=[10,100] # int or None 
featureSelectionUsingModel_list=[True,False]

model=XGBClassifier
modelName='XGBClassifier'

for model_params in param_list:
    for featureSelectionUsingModel in featureSelectionUsingModel_list:
        for k in k_list:
            for n_splits in n_splits_list:
                for n_comp in n_comp_list:
                    for featureSelectionUsingModel in featureSelectionUsingModel_list:
    
                        # (1) general definitions:
                        datasetType='PNP_Cardio' #PNP/PNP_Cardio
                        #nSeqSS=9000 #5000/9000/15000/'all'

                        #(2) model definitions:
                        ResultFolder='%s/TCR_real_data/Predictions' %MyPath
                        if not isdir(ResultFolder):
                            makedirs(ResultFolder)
                        useCV=True
                        stratifiedCV=True

                        #(3) target definition:
                        targetDF=pd.DataFrame(index=['BD'+str(x) for x in range(1,1100)])
                        targetDF['isCardio']=np.where(targetDF.index.str.replace('BD','').astype(int)>949,1,0)
                        targetName='isCardio'
                        YName=targetName

                        #  (4) feature composition:
                        XName='PNP530Cardio126all_phenotypes&PCA&TCRfeatures'
                        ## naming instructions: phenotypes: indicate old or new, TCRdf-indicate threshold to inclusion and binary/RA, 
                        ## PCA- indicate n_comp and is Sparse, indicate feature selection
                        usePhenotype=True #True/False
                        useTCRdf=False #True/False
                        useTCRfeatures=True #True/False
                        usePCAdf=True #True/False

                        # (5) phenotypeDF:

                        genPhenotypeDF=True #True/False. no sense for 'True'  if usePhenotype=False
                        phenotypefile=None
                        phenotypeDFname='newSmall' #('small','allNum and more- see step #6.2


                        # (6) TCRdf:
                        datasetFolder='%s/TCR_real_data/PNP530Cardio126Combined' %MyPath
                        datasetName='PNP530Cardio126' 
                        TCRdfName='sharingMatrix_PNP530Cardio126_minNshared5_RA_onlyProductiveTrue__percShared10_OLtrimmed_binary'

                        #(7) PCAdf (based on the TCRdf selected)

                        isSparse=True #True/False

                        #(8) TCRfeatures:
                        getTCRfeatures=False #True/False. do not use True when useTCRfeatures=False

                        #(9) X and y processing:
                    #     folderToSampleList='%s/SamplesForAnalysis_corrected' %datasetFolder
                    #     sampleListName='matched86List' ################IMPORTANT!!! ###################
                    #     matched86List=gen_sampleList_from_Folder(folderToSampleList,'PNP530Cardio126Matched86')
                        with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/PNP530','rb') as fp:
                            PNP530=pickle.load(fp)
                        with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/Cardio126','rb') as fp:
                            Cardio126=pickle.load(fp)
                        PNP530Cardio126list=PNP530+Cardio126
                        sampleList=PNP530Cardio126list
                        sampleListName='PNP530Cardio126list'

                        filterFeaturesByCorr=None #None or float between 0 and 1
                        C=0.5 #None (if none, C will be set to 0.1) or float between 0 and 1
                        filterFeaturesByUnivariate=f_classif     # None/chi2/f_classif/mutual_info_classif

                        predictionPipeline(datasetType,ResultFolder,model,modelName,model_params,n_splits, useCV,stratifiedCV,
                                              XName,usePhenotype,useTCRdf,useTCRfeatures,usePCAdf,
                                              YName,targetDF,
                                              genPhenotypeDF,phenotypefile,phenotypeDFname,
                                              datasetFolder,datasetName,TCRdfName,n_comp,isSparse,
                                              getTCRfeatures,sampleListName,sampleList,
                                              filterFeaturesByCorr,featureSelectionUsingModel,C,
                                              filterFeaturesByUnivariate,k)



#### get results:

In [ ]:
folder='%s/TCR_real_data/Predictions/isCardio_predictions/%s/predictionDFs' %(MyPath,modelName)
results=concat_summarizing_dfs_excel(folder)
results=results.sort_values(by='roc_auc', ascending=False)
f1='%s/TCR_real_data/Predictions/isCardio_predictions/%s/resultSummary_%s_FullPNP530Cardio126.xlsx' %(MyPath, modelName,cdate)
results.to_excel(f1)
results.head(20)

# predicting isCardio using all ss5000 Samples

## prepare phenotypeDF, TCRdf, TCRfeatureDF

### TCRdf

In [ ]:

# parameters to change:
#general parameters:
DFtype='TCR'
genDF=True #False=generate new df
toBinary=True

#basic prameters for generating df:
mbLevel='g'
useShortName=True

datasetFolder='%s/TCR_real_data/PNP530Cardio126Combined/notMatchedSamples/ss5000rep1' %MyPath
datasetName='PNP530Cardio126_ss5000rep1_notMatched' 

#parameters for manipulations on df 
minVal=None    #minVal can be None,0, float, or 'dfMinVal' or dfMinVal2:
minSharedT=None #minimal number of samples shared by seq/species required to leave sample in the database (int or None)
percShared=10 #minimal number of samples shared by seq/species required to leave sample in the database (int [ percentage]
                #or None)
removeOutliers=True
normData=True
logTransform=True




#constant parameters:
extractUniqueAA=True # use True when this is the first time to analyze this dataset, otherwise, use False
minNshared=2
onlyProductive=True

mbDataFolder='AllSeqProjects'
folder='%s/SamplesForAnalysis_corrected' %datasetFolder
toSaveName='%s/%s_sampleList' %(datasetFolder, datasetName)
SampleList=gen_sampleList_from_Folder(folder,toSaveName=None)
SampleListName='%s_sampleList' %datasetName
libPrepMethod=None
filterGenotek=True
filterMinimalReads=9000000
filterlibPrepMethod=libPrepMethod
groupFunction='noOutlierMean'
nSTD=5
nMinSamples=3
ignoreNotSameYear=True
removeSamePerson=False




df=genTCRorMBdfWithManipulations(DFtype,genDF,toBinary,removeOutliers,normData,logTransform,
                                 minVal,minSharedT,percShared,
                                 mbLevel,useShortName,datasetFolder,datasetName,extractUniqueAA,
                                minNshared,onlyProductive,mbDataFolder,SampleList,
                                  SampleListName,filterlibPrepMethod,filterGenotek, 
                                  groupFunction, nSTD, nMinSamples, ignoreNotSameYear, removeSamePerson)


## run model:

### phenotypes only, try different models:

In [ ]:
modelAndParams_list=[(LogisticRegression,'LogisticRegression',{}),
                   (lgb.LGBMClassifier,'LGBMClassifier',{'learning_rate': 0.1, 'n_estimators': 1000,'max_depth': 3,'num_threads':2}),
                   (XGBClassifier,'XGBClassifier',{'learning_rate': 0.05, 'n_estimators': 100,'max_depth': 6,'num_threads':2}),
                   (GaussianNB,'GaussianNB',{})]


for n,modelAndParams in enumerate(modelAndParams_list):
    
    model=modelAndParams[0]
    modelName=modelAndParams[1]
    model_params=modelAndParams[2]
    
    
    print n,modelName
    
    # (1) general definitions:
    datasetType='PNP_Cardio' #PNP/PNP_Cardio
    #nSeqSS=9000 #5000/9000/15000/'all'

    #(2) model definitions:
    ResultFolder='%s/TCR_real_data/Predictions' %MyPath
    if not isdir(ResultFolder):
        makedirs(ResultFolder)

    n_splits=3
    useCV=True
    stratifiedCV=True

    #(3) target definition:
    targetDF=pd.DataFrame(index=['BD'+str(x) for x in range(1,1100)])
    targetDF['isCardio']=np.where(targetDF.index.str.replace('BD','').astype(int)>949,1,0)
    targetName='isCardio'
    YName=targetName

    #  (4) feature composition:
    XName='PNP530Cardio126ss5000notMatched_phenotypesOnly'
    ## naming instructions: phenotypes: indicate old or new, TCRdf-indicate threshold to inclusion and binary/RA, 
    ## PCA- indicate n_comp and is Sparse, indicate feature selection
    usePhenotype=True #True/False
    useTCRdf=False #True/False
    useTCRfeatures=False #True/False
    usePCAdf=False #True/False

    # (5) phenotypeDF:

    genPhenotypeDF=True #True/False. no sense for 'True'  if usePhenotype=False
    phenotypefile=None
    phenotypeDFname='newSmall' #('small','allNum and more- see step #6.2


    # (6) TCRdf:
    datasetFolder='%s/TCR_real_data/PNP530Cardio126Combined/notMatchedSamples/ss5000rep1' %MyPath
    datasetName='PNP530Cardio126_ss5000rep1_notMatched'  
    TCRdfName='sharingMatrix_PNP530Cardio126_ss5000rep1_notMatched_minNshared2_RA_onlyProductiveTrue__percShared10_OLtrimmed_binary'

    #(7) PCAdf (based on the TCRdf selected)
    n_comp=100 # int or None 
    isSparse=True #True/False

    #(8) TCRfeatures:
    getTCRfeatures=False #True/False. do not use True when useTCRfeatures=False
    TCRfeatureDF=None
    #(9) X and y processing:
    
    #sampleList to filter, and sampleListName:
    mbDataFolder='AllSeqProjects'
    folder='%s/SamplesForAnalysis_corrected' %datasetFolder
    toSaveName='%s_sampleList' %datasetName
    sampleList=gen_sampleList_from_Folder(folder,toSaveName=toSaveName)
    sampleListName='%s_sampleList' %datasetName

    filterFeaturesByCorr=None #None or float between 0 and 1
    featureSelectionUsingModel=False # True/False
    C=0.5 #None (if none, C will be set to 0.1) or float between 0 and 1
    filterFeaturesByUnivariate=f_classif     # None/chi2/f_classif/mutual_info_classif
    k=150 #None (will be set to 50 / int - defines number of k best))


    predictionPipeline (datasetType,ResultFolder,model,modelName,model_params,n_splits, useCV,stratifiedCV,
                          XName,usePhenotype,useTCRdf,useTCRfeatures,usePCAdf,
                          YName,targetDF,
                          genPhenotypeDF,phenotypefile,phenotypeDFname,
                          datasetFolder,datasetName,TCRdfName,n_comp,isSparse,
                          getTCRfeatures,TCRfeatureDF,sampleListName,sampleList,
                          filterFeaturesByCorr,featureSelectionUsingModel,C,
                          filterFeaturesByUnivariate,k)
    

### phenotypes+PCA+TCRdf, try different models

In [ ]:
modelAndParams_list=[(LogisticRegression,'LogisticRegression',{}),
                   (lgb.LGBMClassifier,'LGBMClassifier',{'learning_rate': 0.1, 'n_estimators': 1000,'max_depth': 3,'num_threads':2}),
                   (XGBClassifier,'XGBClassifier',{'learning_rate': 0.05, 'n_estimators': 100,'max_depth': 6,'num_threads':2}),
                   (GaussianNB,'GaussianNB',{})]


for n,modelAndParams in enumerate(modelAndParams_list):
    
    model=modelAndParams[0]
    modelName=modelAndParams[1]
    model_params=modelAndParams[2]
    
    
    print n,modelName
    
    # (1) general definitions:
    datasetType='PNP_Cardio' #PNP/PNP_Cardio
    #nSeqSS=9000 #5000/9000/15000/'all'

    #(2) model definitions:
    ResultFolder='%s/TCR_real_data/Predictions' %MyPath
    if not isdir(ResultFolder):
        makedirs(ResultFolder)

    n_splits=3
    useCV=True
    stratifiedCV=True

    #(3) target definition:
    targetDF=pd.DataFrame(index=['BD'+str(x) for x in range(1,1100)])
    targetDF['isCardio']=np.where(targetDF.index.str.replace('BD','').astype(int)>949,1,0)
    targetName='isCardio'
    YName=targetName

    #  (4) feature composition:
    XName='PNP530Cardio126ss5000notMatched_phenotypes&PCA&TCRfeatures'
    ## naming instructions: phenotypes: indicate old or new, TCRdf-indicate threshold to inclusion and binary/RA, 
    ## PCA- indicate n_comp and is Sparse, indicate feature selection
    usePhenotype=True #True/False
    useTCRdf=False #True/False
    useTCRfeatures=True #True/False
    usePCAdf=True #True/False

    # (5) phenotypeDF:

    genPhenotypeDF=True #True/False. no sense for 'True'  if usePhenotype=False
    phenotypefile=None
    phenotypeDFname='newSmall' #('small','allNum and more- see step #6.2


    # (6) TCRdf:
    datasetFolder='%s/TCR_real_data/PNP530Cardio126Combined/notMatchedSamples/ss5000rep1' %MyPath
    datasetName='PNP530Cardio126_ss5000rep1_notMatched'  
    TCRdfName='sharingMatrix_PNP530Cardio126_ss5000rep1_notMatched_minNshared2_RA_onlyProductiveTrue__percShared10_OLtrimmed_binary'

    
    #(7) PCAdf (based on the TCRdf selected)
    n_comp=10 # int or None 
    isSparse=True #True/False

    #(8) TCRfeatures:
    getTCRfeatures=False #True/False. do not use True when useTCRfeatures=False
    TCRfeatureDF=None
#     PNPfeatureFile='%s/TCR_real_data/PNP530_SubSampled12500data_rep1/featureSummaryDFs/PNP530_ss12500_rep1_allFeatures' %MyPath
#     PNPfeatureDF=pd.read_pickle(PNPfeatureFile)
#     CardiofeatureFile='%s/TCR_real_data/CardioSamples/Cardio126_SubSampled12500data_rep1/featureSummaryDFs/Cardio126_ss12500_rep1_allFeatures' %MyPath
#     CardiofeatureDF=pd.read_pickle(CardiofeatureFile)

#     TCRfeatureDF=pd.concat([PNPfeatureDF,CardiofeatureDF])
                                                                                                                                              
    #(9) X and y processing:
    #sampleList to filter, and sampleListName:
    mbDataFolder='AllSeqProjects'
    folder='%s/SamplesForAnalysis_corrected' %datasetFolder
    toSaveName='%s_sampleList' %datasetName
    sampleList=gen_sampleList_from_Folder(folder,toSaveName=None)
    sampleListName='%s_sampleList' %datasetName

    filterFeaturesByCorr=None #None or float between 0 and 1
    featureSelectionUsingModel=False # True/False
    C=0.5 #None (if none, C will be set to 0.1) or float between 0 and 1
    filterFeaturesByUnivariate=f_classif     # None/chi2/f_classif/mutual_info_classif
    k=150 #None (will be set to 50 / int - defines number of k best))


    predictionPipeline(datasetType,ResultFolder,model,modelName,model_params,n_splits, useCV,stratifiedCV,
                          XName,usePhenotype,useTCRdf,useTCRfeatures,usePCAdf,
                          YName,targetDF,
                          genPhenotypeDF,phenotypefile,phenotypeDFname,
                          datasetFolder,datasetName,TCRdfName,n_comp,isSparse,
                          getTCRfeatures,TCRfeatureDF,sampleListName,sampleList,
                          filterFeaturesByCorr,featureSelectionUsingModel,C,
                          filterFeaturesByUnivariate,k)

    

#### plot results:

In [ ]:
the function plot_modelComparison was copied to predictionFunctions.py

In [ ]:
f1='%s/TCR_real_data/Predictions/isCardio_predictions/modelComparison_180718 - PNP530Cardio126notMatchedss5000.xlsx' %MyPath
resultDF=pd.read_excel(f1)

title='ROC-AUC and PR-AUC (corrected)\nonly-phenotypes vs.\
phenotypes+TCRseqPCA+TCRfeatures\nPNP530 and Cardio126 cohorts - ss5000'

fig=plot_modelComparison(resultDF,title)


### phenotypes+PCA+features, optimize LGBM:

In [ ]:
param_list=[{'learning_rate': 0.1, 'n_estimators': 1000,'max_depth': 3,'num_threads':2},
            {'learning_rate': 0.1, 'n_estimators': 1000,'max_depth': 3,'num_threads':2,
             'bagging_fraction':0.5,'bagging_freq':1, 'bagging_seed':3}]
featureSelectionUsingModel_list=[True,False]
filterFeaturesByUnivariate_list=[(False,None),(f_classif,50), (f_classif,100),(f_classif,150)]
n_splits_list=[3,5,10]

model=lgb.LGBMClassifier
modelName='LGBMClassifier'

for model_params in param_list:
    for featureSelectionUsingModel in featureSelectionUsingModel_list:
        for univariate in filterFeaturesByUnivariate_list:
            filterFeaturesByUnivariate=univariate[0]
            k=univariate[1]
            for n_splits in n_splits_list:
    
                # (1) general definitions:
                datasetType='PNP_Cardio' #PNP/PNP_Cardio
                #nSeqSS=9000 #5000/9000/15000/'all'

                #(2) model definitions:
                ResultFolder='%s/TCR_real_data/Predictions' %MyPath
                if not isdir(ResultFolder):
                    makedirs(ResultFolder)

                useCV=True
                stratifiedCV=True

                #(3) target definition:
                targetDF=pd.DataFrame(index=['BD'+str(x) for x in range(1,1100)])
                targetDF['isCardio']=np.where(targetDF.index.str.replace('BD','').astype(int)>949,1,0)
                targetName='isCardio'
                YName=targetName

                #  (4) feature composition:
                XName='PNP530Cardio126ss5000notMatched_phenotypes&PCA&TCRfeatures'
                ## naming instructions: phenotypes: indicate old or new, TCRdf-indicate threshold to inclusion and binary/RA, 
                ## PCA- indicate n_comp and is Sparse, indicate feature selection
                usePhenotype=True #True/False
                useTCRdf=False #True/False
                useTCRfeatures=True #True/False
                usePCAdf=True #True/False

                # (5) phenotypeDF:

                genPhenotypeDF=True #True/False. no sense for 'True'  if usePhenotype=False
                phenotypefile=None
                phenotypeDFname='newSmall' #('small','allNum and more- see step #6.2


                # (6) TCRdf:
                datasetFolder='%s/TCR_real_data/PNP530Cardio126Combined/notMatchedSamples/ss5000rep1' %MyPath
                datasetName='PNP530Cardio126_ss5000rep1_notMatched'  
                TCRdfName='sharingMatrix_PNP530Cardio126_ss5000rep1_notMatched_minNshared2_RA_onlyProductiveTrue__percShared10_OLtrimmed_binary'


                #(7) PCAdf (based on the TCRdf selected)
                n_comp=100 # int or None 
                isSparse=True #True/False

                #(8) TCRfeatures:
                getTCRfeatures=False #True/False. do not use True when useTCRfeatures=False
                TCRfeatureDF=None
            #     PNPfeatureFile='%s/TCR_real_data/PNP530_SubSampled12500data_rep1/featureSummaryDFs/PNP530_ss12500_rep1_allFeatures' %MyPath
            #     PNPfeatureDF=pd.read_pickle(PNPfeatureFile)
            #     CardiofeatureFile='%s/TCR_real_data/CardioSamples/Cardio126_SubSampled12500data_rep1/featureSummaryDFs/Cardio126_ss12500_rep1_allFeatures' %MyPath
            #     CardiofeatureDF=pd.read_pickle(CardiofeatureFile)

            #     TCRfeatureDF=pd.concat([PNPfeatureDF,CardiofeatureDF])

                #(9) X and y processing:
                #sampleList to filter, and sampleListName:
                mbDataFolder='AllSeqProjects'
                folder='%s/SamplesForAnalysis_corrected' %datasetFolder
                toSaveName='%s_sampleList' %datasetName
                sampleList=gen_sampleList_from_Folder(folder,toSaveName=None)
                sampleListName='%s_sampleList' %datasetName

                filterFeaturesByCorr=None #None or float between 0 and 1
                C=0.5 #None (if none, C will be set to 0.1) or float between 0 and 1
                filterFeaturesByUnivariate=f_classif     # None/chi2/f_classif/mutual_info_classif


                predictionPipeline(datasetType,ResultFolder,model,modelName,model_params,n_splits, useCV,stratifiedCV,
                                      XName,usePhenotype,useTCRdf,useTCRfeatures,usePCAdf,
                                      YName,targetDF,
                                      genPhenotypeDF,phenotypefile,phenotypeDFname,
                                      datasetFolder,datasetName,TCRdfName,n_comp,isSparse,
                                      getTCRfeatures,TCRfeatureDF,sampleListName,sampleList,
                                      filterFeaturesByCorr,featureSelectionUsingModel,C,
                                      filterFeaturesByUnivariate,k)

#### get results:

In [ ]:
folder='%s/TCR_real_data/Predictions/isCardio_predictions/%s/predictionDFs' %(MyPath,modelName)
results=concat_summarizing_dfs_excel(folder)
results=results.sort_values(by='roc_auc', ascending=False)
f1='%s/TCR_real_data/Predictions/isCardio_predictions/%s/resultSummary_%s.xlsx' %(MyPath, modelName,cdate)
results.to_excel(f1)
results.head(20)

### phenotypes+PCA+features, optimize XGB:

In [ ]:
param_list=[{'learning_rate': 0.05, 'n_estimators': 100,'max_depth': 6,'num_threads':2},
            {'learning_rate': 0.05, 'n_estimators': 100,'max_depth': 6,'num_threads':2,
             'bagging_fraction':0.5,'bagging_freq':1, 'bagging_seed':3}]
featureSelectionUsingModel_list=[True,False]
filterFeaturesByUnivariate_list=[(False,None),(f_classif,50), (f_classif,100),(f_classif,150)]
k_list=[50,100,150]
n_splits_list=[3,5,10]


model=XGBClassifier
modelName='XGBClassifier'

for model_params in param_list:
    for featureSelectionUsingModel in featureSelectionUsingModel_list:
        for univariate in filterFeaturesByUnivariate_list:
            filterFeaturesByUnivariate=univariate[0]
            k=univariate[1]
            for n_splits in n_splits_list:

                # (1) general definitions:
                datasetType='PNP_Cardio' #PNP/PNP_Cardio
                #nSeqSS=9000 #5000/9000/15000/'all'

                #(2) model definitions:
                ResultFolder='%s/TCR_real_data/Predictions' %MyPath
                if not isdir(ResultFolder):
                    makedirs(ResultFolder)

                useCV=True
                stratifiedCV=True

                #(3) target definition:
                targetDF=pd.DataFrame(index=['BD'+str(x) for x in range(1,1100)])
                targetDF['isCardio']=np.where(targetDF.index.str.replace('BD','').astype(int)>949,1,0)
                targetName='isCardio'
                YName=targetName

                #  (4) feature composition:
                XName='PNP530Cardio126ss5000notMatched_phenotypes&PCA&TCRfeatures'
                ## naming instructions: phenotypes: indicate old or new, TCRdf-indicate threshold to inclusion and binary/RA, 
                ## PCA- indicate n_comp and is Sparse, indicate feature selection
                usePhenotype=True #True/False
                useTCRdf=False #True/False
                useTCRfeatures=True #True/False
                usePCAdf=True #True/False

                # (5) phenotypeDF:

                genPhenotypeDF=True #True/False. no sense for 'True'  if usePhenotype=False
                phenotypefile=None
                phenotypeDFname='newSmall' #('small','allNum and more- see step #6.2


                # (6) TCRdf:
                datasetFolder='%s/TCR_real_data/PNP530Cardio126Combined/notMatchedSamples/ss5000rep1' %MyPath
                datasetName='PNP530Cardio126_ss5000rep1_notMatched'  
                TCRdfName='sharingMatrix_PNP530Cardio126_ss5000rep1_notMatched_minNshared2_RA_onlyProductiveTrue__percShared10_OLtrimmed_binary'


                #(7) PCAdf (based on the TCRdf selected)
                n_comp=100 # int or None 
                isSparse=True #True/False

                #(8) TCRfeatures:
                getTCRfeatures=False #True/False. do not use True when useTCRfeatures=False
                TCRfeatureDF=None
            #     PNPfeatureFile='%s/TCR_real_data/PNP530_SubSampled12500data_rep1/featureSummaryDFs/PNP530_ss12500_rep1_allFeatures' %MyPath
            #     PNPfeatureDF=pd.read_pickle(PNPfeatureFile)
            #     CardiofeatureFile='%s/TCR_real_data/CardioSamples/Cardio126_SubSampled12500data_rep1/featureSummaryDFs/Cardio126_ss12500_rep1_allFeatures' %MyPath
            #     CardiofeatureDF=pd.read_pickle(CardiofeatureFile)

            #     TCRfeatureDF=pd.concat([PNPfeatureDF,CardiofeatureDF])

                #(9) X and y processing:
                #sampleList to filter, and sampleListName:
                mbDataFolder='AllSeqProjects'
                folder='%s/SamplesForAnalysis_corrected' %datasetFolder
                toSaveName='%s_sampleList' %datasetName
                sampleList=gen_sampleList_from_Folder(folder,toSaveName=None)
                sampleListName='%s_sampleList' %datasetName

                filterFeaturesByCorr=None #None or float between 0 and 1
                C=0.5 #None (if none, C will be set to 0.1) or float between 0 and 1
                filterFeaturesByUnivariate=f_classif     # None/chi2/f_classif/mutual_info_classif


                predictionPipeline(datasetType,ResultFolder,model,modelName,model_params,n_splits, useCV,stratifiedCV,
                                      XName,usePhenotype,useTCRdf,useTCRfeatures,usePCAdf,
                                      YName,targetDF,
                                      genPhenotypeDF,phenotypefile,phenotypeDFname,
                                      datasetFolder,datasetName,TCRdfName,n_comp,isSparse,
                                      getTCRfeatures,TCRfeatureDF,sampleListName,sampleList,
                                      filterFeaturesByCorr,featureSelectionUsingModel,C,
                                      filterFeaturesByUnivariate,k)

#### get results:

In [ ]:
folder='%s/TCR_real_data/Predictions/isCardio_predictions/%s/predictionDFs' %(MyPath,modelName)
results=concat_summarizing_dfs_excel(folder)
results=results.sort_values(by='roc_auc', ascending=False)
f1='%s/TCR_real_data/Predictions/isCardio_predictions/%s/resultSummary_%s.xlsx' %(MyPath, modelName,cdate)
results.to_excel(f1)
results.head(20)

# predicting isCardio using all ss12500 Samples

## prepare phenotypeDF, TCRdf, TCRfeatureDF

### TCRdf

In [ ]:

# parameters to change:
#general parameters:
DFtype='TCR'
genDF=True #False=generate new df
toBinary=True

#basic prameters for generating df:
mbLevel='g'
useShortName=True

datasetFolder='%s/TCR_real_data/PNP530Cardio126Combined/notMatchedSamples/ss12500rep1' %MyPath
datasetName='PNP530Cardio126_ss12500rep1_notMatched' 

#parameters for manipulations on df 
minVal=None    #minVal can be None,0, float, or 'dfMinVal' or dfMinVal2:
minSharedT=None #minimal number of samples shared by seq/species required to leave sample in the database (int or None)
percShared=10 #minimal number of samples shared by seq/species required to leave sample in the database (int [ percentage]
                #or None)
removeOutliers=True
normData=True
logTransform=True




#constant parameters:
extractUniqueAA=True # use True when this is the first time to analyze this dataset, otherwise, use False
minNshared=2
onlyProductive=True

mbDataFolder='AllSeqProjects'
folder='%s/SamplesForAnalysis_corrected' %datasetFolder
toSaveName='%s/%s_sampleList' %(datasetFolder, datasetName)
SampleList=gen_sampleList_from_Folder(folder,toSaveName=None)
SampleListName='%s_sampleList' %datasetName
libPrepMethod=None
filterGenotek=True
filterMinimalReads=9000000
filterlibPrepMethod=libPrepMethod
groupFunction='noOutlierMean'
nSTD=5
nMinSamples=3
ignoreNotSameYear=True
removeSamePerson=False




df=genTCRorMBdfWithManipulations(DFtype,genDF,toBinary,removeOutliers,normData,logTransform,
                                 minVal,minSharedT,percShared,
                                 mbLevel,useShortName,datasetFolder,datasetName,extractUniqueAA,
                                minNshared,onlyProductive,mbDataFolder,SampleList,
                                  SampleListName,filterlibPrepMethod,filterGenotek, 
                                  groupFunction, nSTD, nMinSamples, ignoreNotSameYear, removeSamePerson)


## run model:

### phenotypes only, try different models:

In [ ]:
modelAndParams_list=[(LogisticRegression,'LogisticRegression',{}),
                   (lgb.LGBMClassifier,'LGBMClassifier',{'learning_rate': 0.1, 'n_estimators': 1000,'max_depth': 3,'num_threads':2}),
                   (XGBClassifier,'XGBClassifier',{'learning_rate': 0.05, 'n_estimators': 100,'max_depth': 6,'num_threads':2}),
                   (GaussianNB,'GaussianNB',{})]


for n,modelAndParams in enumerate(modelAndParams_list):
    
    model=modelAndParams[0]
    modelName=modelAndParams[1]
    model_params=modelAndParams[2]
    
    
    print n,modelName
    
    # (1) general definitions:
    datasetType='PNP_Cardio' #PNP/PNP_Cardio
    #nSeqSS=9000 #12500/9000/112500/'all'

    #(2) model definitions:
    ResultFolder='%s/TCR_real_data/Predictions' %MyPath
    if not isdir(ResultFolder):
        makedirs(ResultFolder)

    n_splits=3
    useCV=True
    stratifiedCV=True

    #(3) target definition:
    targetDF=pd.DataFrame(index=['BD'+str(x) for x in range(1,1100)])
    targetDF['isCardio']=np.where(targetDF.index.str.replace('BD','').astype(int)>949,1,0)
    targetName='isCardio'
    YName=targetName

    #  (4) feature composition:
    XName='PNP530Cardio126ss12500notMatched_phenotypesOnly'
    ## naming instructions: phenotypes: indicate old or new, TCRdf-indicate threshold to inclusion and binary/RA, 
    ## PCA- indicate n_comp and is Sparse, indicate feature selection
    usePhenotype=True #True/False
    useTCRdf=False #True/False
    useTCRfeatures=False #True/False
    usePCAdf=False #True/False

    # (5) phenotypeDF:

    genPhenotypeDF=True #True/False. no sense for 'True'  if usePhenotype=False
    phenotypefile=None
    phenotypeDFname='newSmall' #('small','allNum and more- see step #6.2


    # (6) TCRdf:
    datasetFolder='%s/TCR_real_data/PNP530Cardio126Combined/notMatchedSamples/ss12500rep1' %MyPath
    datasetName='PNP530Cardio126_ss12500rep1_notMatched'  
    TCRdfName='sharingMatrix_PNP530Cardio126_ss12500rep1_notMatched_minNshared2_RA_onlyProductiveTrue__percShared10_OLtrimmed_binary'

    #(7) PCAdf (based on the TCRdf selected)
    n_comp=100 # int or None 
    isSparse=True #True/False

    #(8) TCRfeatures:
    getTCRfeatures=True #True/False. do not use True when useTCRfeatures=False
    TCRfeatureDF=None
    #(9) X and y processing:
    
    #sampleList to filter, and sampleListName:
    mbDataFolder='AllSeqProjects'
    folder='%s/SamplesForAnalysis_corrected' %datasetFolder
    toSaveName='%s_sampleList' %datasetName
    sampleList=gen_sampleList_from_Folder(folder,toSaveName=toSaveName)
    sampleListName='%s_sampleList' %datasetName

    filterFeaturesByCorr=None #None or float between 0 and 1
    featureSelectionUsingModel=False # True/False
    C=0.5 #None (if none, C will be set to 0.1) or float between 0 and 1
    filterFeaturesByUnivariate=f_classif     # None/chi2/f_classif/mutual_info_classif
    k=150 #None (will be set to 50 / int - defines number of k best))


    predictionPipeline (datasetType,ResultFolder,model,modelName,model_params,n_splits, useCV,stratifiedCV,
                          XName,usePhenotype,useTCRdf,useTCRfeatures,usePCAdf,
                          YName,targetDF,
                          genPhenotypeDF,phenotypefile,phenotypeDFname,
                          datasetFolder,datasetName,TCRdfName,n_comp,isSparse,
                          getTCRfeatures,TCRfeatureDF,sampleListName,sampleList,
                          filterFeaturesByCorr,featureSelectionUsingModel,C,
                          filterFeaturesByUnivariate,k)
    

### phenotypes+PCA+TCRdf, try different models

In [ ]:
modelAndParams_list=[(LogisticRegression,'LogisticRegression',{}),
                   (lgb.LGBMClassifier,'LGBMClassifier',{'learning_rate': 0.1, 'n_estimators': 1000,'max_depth': 3,'num_threads':2}),
                   (XGBClassifier,'XGBClassifier',{'learning_rate': 0.05, 'n_estimators': 100,'max_depth': 6,'num_threads':2}),
                   (GaussianNB,'GaussianNB',{})]


for n,modelAndParams in enumerate(modelAndParams_list):
    
    model=modelAndParams[0]
    modelName=modelAndParams[1]
    model_params=modelAndParams[2]
    
    
    print n,modelName
    
    # (1) general definitions:
    datasetType='PNP_Cardio' #PNP/PNP_Cardio
    #nSeqSS=9000 #12500/9000/112500/'all'

    #(2) model definitions:
    ResultFolder='%s/TCR_real_data/Predictions' %MyPath
    if not isdir(ResultFolder):
        makedirs(ResultFolder)

    n_splits=3
    useCV=True
    stratifiedCV=True

    #(3) target definition:
    targetDF=pd.DataFrame(index=['BD'+str(x) for x in range(1,1100)])
    targetDF['isCardio']=np.where(targetDF.index.str.replace('BD','').astype(int)>949,1,0)
    targetName='isCardio'
    YName=targetName

    #  (4) feature composition:
    XName='PNP530Cardio126ss12500notMatched_phenotypes&PCA&TCRfeatures'
    ## naming instructions: phenotypes: indicate old or new, TCRdf-indicate threshold to inclusion and binary/RA, 
    ## PCA- indicate n_comp and is Sparse, indicate feature selection
    usePhenotype=True #True/False
    useTCRdf=False #True/False
    useTCRfeatures=True #True/False
    usePCAdf=True #True/False

    # (5) phenotypeDF:

    genPhenotypeDF=True #True/False. no sense for 'True'  if usePhenotype=False
    phenotypefile=None
    phenotypeDFname='newSmall' #('small','allNum and more- see step #6.2


    # (6) TCRdf:
    datasetFolder='%s/TCR_real_data/PNP530Cardio126Combined/notMatchedSamples/ss12500rep1' %MyPath
    datasetName='PNP530Cardio126_ss12500rep1_notMatched'  
    TCRdfName='sharingMatrix_PNP530Cardio126_ss12500rep1_notMatched_minNshared2_RA_onlyProductiveTrue__percShared10_OLtrimmed_binary'

    
    #(7) PCAdf (based on the TCRdf selected)
    n_comp=10 # int or None 
    isSparse=True #True/False

    #(8) TCRfeatures:
    getTCRfeatures=False #True/False. do not use True when useTCRfeatures=False
    TCRfeatureDF=None
#     PNPfeatureFile='%s/TCR_real_data/PNP530_SubSampled12500data_rep1/featureSummaryDFs/PNP530_ss12500_rep1_allFeatures' %MyPath
#     PNPfeatureDF=pd.read_pickle(PNPfeatureFile)
#     CardiofeatureFile='%s/TCR_real_data/CardioSamples/Cardio126_SubSampled12500data_rep1/featureSummaryDFs/Cardio126_ss12500_rep1_allFeatures' %MyPath
#     CardiofeatureDF=pd.read_pickle(CardiofeatureFile)

#     TCRfeatureDF=pd.concat([PNPfeatureDF,CardiofeatureDF])
                                                                                                                                              
    #(9) X and y processing:
    #sampleList to filter, and sampleListName:
    mbDataFolder='AllSeqProjects'
    folder='%s/SamplesForAnalysis_corrected' %datasetFolder
    toSaveName='%s_sampleList' %datasetName
    sampleList=gen_sampleList_from_Folder(folder,toSaveName=None)
    sampleListName='%s_sampleList' %datasetName

    filterFeaturesByCorr=None #None or float between 0 and 1
    featureSelectionUsingModel=False # True/False
    C=0.5 #None (if none, C will be set to 0.1) or float between 0 and 1
    filterFeaturesByUnivariate=f_classif     # None/chi2/f_classif/mutual_info_classif
    k=150 #None (will be set to 50 / int - defines number of k best))


    predictionPipeline(datasetType,ResultFolder,model,modelName,model_params,n_splits, useCV,stratifiedCV,
                          XName,usePhenotype,useTCRdf,useTCRfeatures,usePCAdf,
                          YName,targetDF,
                          genPhenotypeDF,phenotypefile,phenotypeDFname,
                          datasetFolder,datasetName,TCRdfName,n_comp,isSparse,
                          getTCRfeatures,TCRfeatureDF,sampleListName,sampleList,
                          filterFeaturesByCorr,featureSelectionUsingModel,C,
                          filterFeaturesByUnivariate,k)

    

#### plot results:

In [ ]:
f1='%s/TCR_real_data/Predictions/isCardio_predictions/modelComparison_180718 - PNP530Cardio126notMatchedss12500.xlsx' %MyPath
resultDF=pd.read_excel(f1)

title='ROC-AUC and PR-AUC (corrected)\nonly-phenotypes vs.\
phenotypes+TCRseqPCA+TCRfeatures\nPNP530 and Cardio126 cohorts - ss12500'

fig=plot_modelComparison(resultDF,title)


### phenotypes+PCA+features, optimize LOGISTIC REGRESSION:

In [ ]:
penalty_list=['l1','l2']
C_list=[0.1,1,10]
fit_intercept_list=[True,False]
featureSelectionUsingModel_list=[True,False]
filterFeaturesByUnivariate_list=[(False,None),(f_classif,50), (f_classif,100),(f_classif,150)]
n_splits_list=[3,5,10]

param_list=[{'learning_rate': 0.05, 'n_estimators': 100,'max_depth': 6,'num_threads':2},
            {'learning_rate': 0.05, 'n_estimators': 100,'max_depth': 6,'num_threads':2,
             'bagging_fraction':0.5,'bagging_freq':1, 'bagging_seed':3}]


model=LogisticRegression
modelName='LogisticRegression'

for penalty in penalty_list:
    for C in C_list:
        for fit_intercept in fit_intercept_list:
            model_params={'penalty':penalty,'C':C,'fit_intercept':fit_intercept}
            for featureSelectionUsingModel in featureSelectionUsingModel_list:
                for univariate in filterFeaturesByUnivariate_list:
                    filterFeaturesByUnivariate=univariate[0]
                    k=univariate[1]
                    for n_splits in n_splits_list:

                        # (1) general definitions:
                        datasetType='PNP_Cardio' #PNP/PNP_Cardio
                        #nSeqSS=9000 #12500/9000/112500/'all'

                        #(2) model definitions:
                        ResultFolder='%s/TCR_real_data/Predictions' %MyPath
                        if not isdir(ResultFolder):
                            makedirs(ResultFolder)

                        useCV=True
                        stratifiedCV=True

                        #(3) target definition:
                        targetDF=pd.DataFrame(index=['BD'+str(x) for x in range(1,1100)])
                        targetDF['isCardio']=np.where(targetDF.index.str.replace('BD','').astype(int)>949,1,0)
                        targetName='isCardio'
                        YName=targetName

                        #  (4) feature composition:
                        XName='PNP530Cardio126ss12500notMatched_phenotypes&PCA&TCRfeatures'
                        ## naming instructions: phenotypes: indicate old or new, TCRdf-indicate threshold to inclusion and binary/RA, 
                        ## PCA- indicate n_comp and is Sparse, indicate feature selection
                        usePhenotype=True #True/False
                        useTCRdf=False #True/False
                        useTCRfeatures=True #True/False
                        usePCAdf=True #True/False

                        # (5) phenotypeDF:

                        genPhenotypeDF=True #True/False. no sense for 'True'  if usePhenotype=False
                        phenotypefile=None
                        phenotypeDFname='newSmall' #('small','allNum and more- see step #6.2


                        # (6) TCRdf:
                        datasetFolder='%s/TCR_real_data/PNP530Cardio126Combined/notMatchedSamples/ss12500rep1' %MyPath
                        datasetName='PNP530Cardio126_ss12500rep1_notMatched'  
                        TCRdfName='sharingMatrix_PNP530Cardio126_ss12500rep1_notMatched_minNshared2_RA_onlyProductiveTrue__percShared10_OLtrimmed_binary'


                        #(7) PCAdf (based on the TCRdf selected)
                        n_comp=100 # int or None 
                        isSparse=True #True/False

                        #(8) TCRfeatures:
                        getTCRfeatures=False #True/False. do not use True when useTCRfeatures=False
                        TCRfeatureDF=None
                    #     PNPfeatureFile='%s/TCR_real_data/PNP530_SubSampled12500data_rep1/featureSummaryDFs/PNP530_ss12500_rep1_allFeatures' %MyPath
                    #     PNPfeatureDF=pd.read_pickle(PNPfeatureFile)
                    #     CardiofeatureFile='%s/TCR_real_data/CardioSamples/Cardio126_SubSampled12500data_rep1/featureSummaryDFs/Cardio126_ss12500_rep1_allFeatures' %MyPath
                    #     CardiofeatureDF=pd.read_pickle(CardiofeatureFile)

                    #     TCRfeatureDF=pd.concat([PNPfeatureDF,CardiofeatureDF])

                        #(9) X and y processing:
                        #sampleList to filter, and sampleListName:
                        mbDataFolder='AllSeqProjects'
                        folder='%s/SamplesForAnalysis_corrected' %datasetFolder
                        toSaveName='%s_sampleList' %datasetName
                        sampleList=gen_sampleList_from_Folder(folder,toSaveName=None)
                        sampleListName='%s_sampleList' %datasetName

                        filterFeaturesByCorr=None #None or float between 0 and 1
#                         C=0.5 #None (if none, C will be set to 0.1) or float between 0 and 1
                        filterFeaturesByUnivariate=f_classif     # None/chi2/f_classif/mutual_info_classif


                        predictionPipeline(datasetType,ResultFolder,model,modelName,model_params,n_splits, useCV,stratifiedCV,
                                              XName,usePhenotype,useTCRdf,useTCRfeatures,usePCAdf,
                                              YName,targetDF,
                                              genPhenotypeDF,phenotypefile,phenotypeDFname,
                                              datasetFolder,datasetName,TCRdfName,n_comp,isSparse,
                                              getTCRfeatures,TCRfeatureDF,sampleListName,sampleList,
                                              filterFeaturesByCorr,featureSelectionUsingModel,C,
                                              filterFeaturesByUnivariate,k)

#### get results:

In [ ]:
folder='%s/TCR_real_data/Predictions/isCardio_predictions/%s/predictionDFs' %(MyPath,modelName)
results=concat_summarizing_dfs_excel(folder)
results=results.sort_values(by='roc_auc', ascending=False)
f1='%s/TCR_real_data/Predictions/isCardio_predictions/%s/resultSummary_%s.xlsx' %(MyPath, modelName,cdate)
results.to_excel(f1)
results.head(20)

# compare models and cohorts for the additional information gained by TCR data using best optimized parameters for each model

## ss12500:

In [ ]:
onlyPhen_list=[True,False]
modelAndParams_list=[(LogisticRegression,'LogisticRegression',{'C':10,'penalty':'l1'}),
                   (lgb.LGBMClassifier,'LGBMClassifier',{'learning_rate': 0.1, 'n_estimators': 1000,'max_depth': 3,'num_threads':2,
             'bagging_fraction':0.5,'bagging_freq':1, 'bagging_seed':3}),
                   (XGBClassifier,'XGBClassifier',{'learning_rate': 0.05, 'n_estimators': 100,'max_depth': 6,'num_threads':2,
             'bagging_fraction':0.5,'bagging_freq':1, 'bagging_seed':3})]
scale_list=[True,False]



for n,modelAndParams in enumerate(modelAndParams_list):
    for scale in scale_list:
        for onlyPhen in onlyPhen_list:
            if onlyPhen:
                XName='PNP530Cardio126ss12500notMatched_phenotypesOnly'
                useTCRfeatures=False #True/False
                usePCAdf=False #True/False    
            else:
                XName='PNP530Cardio126ss12500notMatched_phenotypes&PCA&TCRfeatures'
                useTCRfeatures=True #True/False
                usePCAdf=True #True/False
    
            model=modelAndParams[0]
            modelName=modelAndParams[1]
            model_params=modelAndParams[2]


            print n,modelName

            # (1) general definitions:
            datasetType='PNP_Cardio' #PNP/PNP_Cardio
            #nSeqSS=9000 #12500/9000/112500/'all'

            #(2) model definitions:
            ResultFolder='%s/TCR_real_data/Predictions' %MyPath
            if not isdir(ResultFolder):
                makedirs(ResultFolder)

            n_splits=10
            useCV=True
            stratifiedCV=True

            #(3) target definition:
            targetDF=pd.DataFrame(index=['BD'+str(x) for x in range(1,1100)])
            targetDF['isCardio']=np.where(targetDF.index.str.replace('BD','').astype(int)>949,1,0)
            targetName='isCardio'
            YName=targetName

            #  (4) feature composition:

            ## naming instructions: phenotypes: indicate old or new, TCRdf-indicate threshold to inclusion and binary/RA, 
            ## PCA- indicate n_comp and is Sparse, indicate feature selection
            usePhenotype=True #True/False
            useTCRdf=False #True/False

            # (5) phenotypeDF:

            genPhenotypeDF=True #True/False. no sense for 'True'  if usePhenotype=False
            phenotypefile=None
            phenotypeDFname='newSmall' #('small','allNum and more- see step #6.2


            # (6) TCRdf:
            datasetFolder='%s/TCR_real_data/PNP530Cardio126Combined/notMatchedSamples/ss12500rep1' %MyPath
            datasetName='PNP530Cardio126_ss12500rep1_notMatched'  
            TCRdfName='sharingMatrix_PNP530Cardio126_ss12500rep1_notMatched_minNshared2_RA_onlyProductiveTrue__percShared10_OLtrimmed_binary'


            #(7) PCAdf (based on the TCRdf selected)
            n_comp=100 # int or None 
            isSparse=True #True/False

            #(8) TCRfeatures:
            getTCRfeatures=False #True/False. do not use True when useTCRfeatures=False
            TCRfeatureDF=None
        #     PNPfeatureFile='%s/TCR_real_data/PNP530_SubSampled12500data_rep1/featureSummaryDFs/PNP530_ss12500_rep1_allFeatures' %MyPath
        #     PNPfeatureDF=pd.read_pickle(PNPfeatureFile)
        #     CardiofeatureFile='%s/TCR_real_data/CardioSamples/Cardio126_SubSampled12500data_rep1/featureSummaryDFs/Cardio126_ss12500_rep1_allFeatures' %MyPath
        #     CardiofeatureDF=pd.read_pickle(CardiofeatureFile)

        #     TCRfeatureDF=pd.concat([PNPfeatureDF,CardiofeatureDF])

            #(9) X and y processing:
            #sampleList to filter, and sampleListName:
            mbDataFolder='AllSeqProjects'
            folder='%s/SamplesForAnalysis_corrected' %datasetFolder
            toSaveName='%s_sampleList' %datasetName
            sampleList=gen_sampleList_from_Folder(folder,toSaveName=None)
            sampleListName='%s_sampleList' %datasetName

            filterFeaturesByCorr=None #None or float between 0 and 1
            featureSelectionUsingModel=False # True/False
            C=0.5 #None (if none, C will be set to 0.1) or float between 0 and 1
            filterFeaturesByUnivariate=f_classif     # None/chi2/f_classif/mutual_info_classif
            k=50 #None (will be set to 50 / int - defines number of k best))
            print 'scale=%s' %scale

            predictionPipeline(datasetType,ResultFolder,model,modelName,model_params,n_splits, useCV,stratifiedCV,
                                  XName,usePhenotype,useTCRdf,useTCRfeatures,usePCAdf,
                                  YName,targetDF,
                                  genPhenotypeDF,phenotypefile,phenotypeDFname,
                                  datasetFolder,datasetName,TCRdfName,n_comp,isSparse,
                                  getTCRfeatures,TCRfeatureDF,sampleListName,sampleList,
                                  filterFeaturesByCorr,featureSelectionUsingModel,scale,C,
                                  filterFeaturesByUnivariate,k)



## ss5000:

In [ ]:
onlyPhen_list=[True,False]
modelAndParams_list=[(LogisticRegression,'LogisticRegression',{'C':10,'penalty':'l1'}),
                   (lgb.LGBMClassifier,'LGBMClassifier',{'learning_rate': 0.1, 'n_estimators': 1000,'max_depth': 3,'num_threads':2,
             'bagging_fraction':0.5,'bagging_freq':1, 'bagging_seed':3}),
                   (XGBClassifier,'XGBClassifier',{'learning_rate': 0.05, 'n_estimators': 100,'max_depth': 6,'num_threads':2,
             'bagging_fraction':0.5,'bagging_freq':1, 'bagging_seed':3})]
scale_list=[True,False]



for n,modelAndParams in enumerate(modelAndParams_list):
    for scale in scale_list:
        for onlyPhen in onlyPhen_list:
            if onlyPhen:
                XName='PNP530Cardio126ss5000notMatched_phenotypesOnly'
                useTCRfeatures=False #True/False
                usePCAdf=False #True/False    
            else:
                XName='PNP530Cardio126ss5000notMatched_phenotypes&PCA&TCRfeatures'
                useTCRfeatures=True #True/False
                usePCAdf=True #True/False

            model=modelAndParams[0]
            modelName=modelAndParams[1]
            model_params=modelAndParams[2]


            print n,modelName

            # (1) general definitions:
            datasetType='PNP_Cardio' #PNP/PNP_Cardio
            #nSeqSS=9000 #5000/9000/15000/'all'

            #(2) model definitions:
            ResultFolder='%s/TCR_real_data/Predictions' %MyPath
            if not isdir(ResultFolder):
                makedirs(ResultFolder)

            n_splits=10
            useCV=True
            stratifiedCV=True

            #(3) target definition:
            targetDF=pd.DataFrame(index=['BD'+str(x) for x in range(1,1100)])
            targetDF['isCardio']=np.where(targetDF.index.str.replace('BD','').astype(int)>949,1,0)
            targetName='isCardio'
            YName=targetName

            #  (4) feature composition:

            ## naming instructions: phenotypes: indicate old or new, TCRdf-indicate threshold to inclusion and binary/RA, 
            ## PCA- indicate n_comp and is Sparse, indicate feature selection
            usePhenotype=True #True/False
            useTCRdf=False #True/False

            # (5) phenotypeDF:

            genPhenotypeDF=True #True/False. no sense for 'True'  if usePhenotype=False
            phenotypefile=None
            phenotypeDFname='newSmall' #('small','allNum and more- see step #6.2


            # (6) TCRdf:
            datasetFolder='%s/TCR_real_data/PNP530Cardio126Combined/notMatchedSamples/ss5000rep1' %MyPath
            datasetName='PNP530Cardio126_ss5000rep1_notMatched'  
            TCRdfName='sharingMatrix_PNP530Cardio126_ss5000rep1_notMatched_minNshared2_RA_onlyProductiveTrue__percShared10_OLtrimmed_binary'


            #(7) PCAdf (based on the TCRdf selected)
            n_comp=100 # int or None 
            isSparse=True #True/False

            #(8) TCRfeatures:
            getTCRfeatures=False #True/False. do not use True when useTCRfeatures=False
            TCRfeatureDF=None
        #     PNPfeatureFile='%s/TCR_real_data/PNP530_SubSampled5000data_rep1/featureSummaryDFs/PNP530_ss5000_rep1_allFeatures' %MyPath
        #     PNPfeatureDF=pd.read_pickle(PNPfeatureFile)
        #     CardiofeatureFile='%s/TCR_real_data/CardioSamples/Cardio126_SubSampled5000data_rep1/featureSummaryDFs/Cardio126_ss5000_rep1_allFeatures' %MyPath
        #     CardiofeatureDF=pd.read_pickle(CardiofeatureFile)

        #     TCRfeatureDF=pd.concat([PNPfeatureDF,CardiofeatureDF])

            #(9) X and y processing:
            #sampleList to filter, and sampleListName:
            mbDataFolder='AllSeqProjects'
            folder='%s/SamplesForAnalysis_corrected' %datasetFolder
            toSaveName='%s_sampleList' %datasetName
            sampleList=gen_sampleList_from_Folder(folder,toSaveName=None)
            sampleListName='%s_sampleList' %datasetName

            filterFeaturesByCorr=None #None or float between 0 and 1
            featureSelectionUsingModel=False # True/False
            C=0.5 #None (if none, C will be set to 0.1) or float between 0 and 1
            filterFeaturesByUnivariate=f_classif     # None/chi2/f_classif/mutual_info_classif
            k=50 #None (will be set to 50 / int - defines number of k best))


            predictionPipeline(datasetType,ResultFolder,model,modelName,model_params,n_splits, useCV,stratifiedCV,
                                  XName,usePhenotype,useTCRdf,useTCRfeatures,usePCAdf,
                                  YName,targetDF,
                                  genPhenotypeDF,phenotypefile,phenotypeDFname,
                                  datasetFolder,datasetName,TCRdfName,n_comp,isSparse,
                                  getTCRfeatures,TCRfeatureDF,sampleListName,sampleList,
                                  filterFeaturesByCorr,featureSelectionUsingModel,scale,C,
                                  filterFeaturesByUnivariate,k)



## all sequences:

In [ ]:
onlyPhen_list=[True,False]
modelAndParams_list=[(LogisticRegression,'LogisticRegression',{'C':10,'penalty':'l1'}),
                   (lgb.LGBMClassifier,'LGBMClassifier',{'learning_rate': 0.1, 'n_estimators': 1000,'max_depth': 3,'num_threads':2,
             'bagging_fraction':0.5,'bagging_freq':1, 'bagging_seed':3}),
                   (XGBClassifier,'XGBClassifier',{'learning_rate': 0.05, 'n_estimators': 100,'max_depth': 6,'num_threads':2,
             'bagging_fraction':0.5,'bagging_freq':1, 'bagging_seed':3})]
scale_list=[True,False]



for n,modelAndParams in enumerate(modelAndParams_list):
    for scale in scale_list:
        for onlyPhen in onlyPhen_list:
            if onlyPhen:
                XName='PNP530Cardio126_phenotypesOnly'
                useTCRfeatures=False #True/False
                usePCAdf=False #True/False    
            else:
                XName='PNP530Cardio126_phenotypes&PCA&TCRfeatures'
                useTCRfeatures=True #True/False
                usePCAdf=True #True/False
    
            model=modelAndParams[0]
            modelName=modelAndParams[1]
            model_params=modelAndParams[2]


            print n,modelName

            # (1) general definitions:
            datasetType='PNP_Cardio' #PNP/PNP_Cardio
            #nSeqSS=9000 #5000/9000/15000/'all'

            #(2) model definitions:
            ResultFolder='%s/TCR_real_data/Predictions' %MyPath
            if not isdir(ResultFolder):
                makedirs(ResultFolder)

            n_splits=10
            useCV=True
            stratifiedCV=True

            #(3) target definition:
            targetDF=pd.DataFrame(index=['BD'+str(x) for x in range(1,1100)])
            targetDF['isCardio']=np.where(targetDF.index.str.replace('BD','').astype(int)>949,1,0)
            targetName='isCardio'
            YName=targetName

            #  (4) feature composition:

            ## naming instructions: phenotypes: indicate old or new, TCRdf-indicate threshold to inclusion and binary/RA, 
            ## PCA- indicate n_comp and is Sparse, indicate feature selection
            usePhenotype=True #True/False
            useTCRdf=False #True/False

            # (5) phenotypeDF:

            genPhenotypeDF=True #True/False. no sense for 'True'  if usePhenotype=False
            phenotypefile=None
            phenotypeDFname='newSmall' #('small','allNum and more- see step #6.2


            # (6) TCRdf:
            datasetFolder='%s/TCR_real_data/PNP530Cardio126Combined' %MyPath
            datasetName='PNP530Cardio126'  
            TCRdfName='sharingMatrix_PNP530Cardio126_minNshared5_RA_onlyProductiveTrue__percShared10_OLtrimmed_binary'


            #(7) PCAdf (based on the TCRdf selected)
            n_comp=100 # int or None 
            isSparse=True #True/False

            #(8) TCRfeatures:
            getTCRfeatures=False #True/False. do not use True when useTCRfeatures=False
            TCRfeatureDF=None

            #(9) X and y processing:
            #sampleList to filter, and sampleListName:
            mbDataFolder='AllSeqProjects'
            folder='%s/SamplesForAnalysis_corrected' %datasetFolder
            toSaveName='%s_sampleList' %datasetName
            sampleList=gen_sampleList_from_Folder(folder,toSaveName=None)
            sampleListName='%s_sampleList' %datasetName

            filterFeaturesByCorr=None #None or float between 0 and 1
#                     featureSelectionUsingModel=False # True/False
            #                     C=0.5 #None (if none, C will be set to 0.1) or float between 0 and 1
            #                     filterFeaturesByUnivariate=f_classif    # None/chi2/f_classif/mutual_info_classif
            #             k=150 #None (will be set to 50 / int - defines number of k best))
                    scale=True

                    predictionPipeline(datasetType,ResultFolder,model,modelName,model_params,n_splits, useCV,
                                          stratifiedCV, XName,usePhenotype,useTCRdf,useTCRfeatures,usePCAdf,
                                          YName,targetDF,
                                          genPhenotypeDF,phenotypeDF,phenotypeDFname,
                                          datasetFolder,datasetNameTCRdf,TCRdf,TCRdfName,n_comp,isSparse,
                                          getTCRfeatures,TCRfeatureDF,sampleListName,sampleList,
                                          filterFeaturesByCorr,featureSelectionMethod,scale,
                                          k,cdate=cdate,overwriteResults=True)


In [ ]:
modelName_list=['LogisticRegression','LGBMClassifier','XGBClassifier']

for modelName in modelName_list:
    print modelName
    folder='%s/TCR_real_data/Predictions/isCardio_predictions/%s/predictionDFs' %(MyPath,modelName)
    results=concat_summarizing_dfs_excel(folder)
    results=results.sort_values(by='roc_auc', ascending=False)
    f1='%s/TCR_real_data/Predictions/isCardio_predictions/%s/resultSummary_%s.xlsx' %(MyPath, modelName,cdate)
    results.to_excel(f1)
    results.head(20)

## plot all results for optimized models:

In [ ]:
modelName_list=['LogisticRegression','LGBMClassifier','XGBClassifier']

name_list=['notMatchedss5000','notMatchedss12500','']

for n,name in enumerate(name_list):
    
    f1='%s/TCR_real_data/Predictions/isCardio_predictions/\
modelComparison_180718 - PNP530Cardio126%s - Optimized.xlsx' %(MyPath, name)
    resultDF=pd.read_excel(f1)
        
    title='ROC-AUC and PR-AUC (corrected)\nonly-phenotypes vs.\
    phenotypes+TCRseqPCA+TCRfeatures\nPNP530Cardio126%s' %name

    fig,sumDF=plot_modelComparison(resultDF,title)
    sumDF['ss']=name.replace('notMatcheds','')
    resultDF['ss']=name.replace('notMatcheds','')
    if n==0:
        allSumDF=sumDF
        allResultDF=resultDF
    else:
        allSumDF=pd.concat([allSumDF,sumDF])
        allResultDF=pd.concat([allResultDF,resultDF])
    
print allResultDF.sort_values(by='roc_auc',ascending=False)
print allResultDF.sort_values(by='pr_auc_corrected',ascending=False)
print allSumDF .sort_values(by='ROCdif',ascending=False)
print allSumDF .sort_values(by='PRdif',ascending=False)

    

# ss12500 matched cohort:

## TCRdf

In [ ]:

# parameters to change:
#general parameters:
DFtype='TCR'
genDF=True #False=generate new df
toBinary=True

#basic prameters for generating df:
mbLevel='g'
useShortName=True

datasetFolder='%s/TCR_real_data/PNP530Cardio126Combined/MatchedSamples/ss12500rep1' %MyPath
datasetName='PNP530Cardio126_ss12500rep1_Matched' 

#parameters for manipulations on df 
minVal=None    #minVal can be None,0, float, or 'dfMinVal' or dfMinVal2:
minSharedT=None #minimal number of samples shared by seq/species required to leave sample in the database (int or None)
percShared=10 #minimal number of samples shared by seq/species required to leave sample in the database (int [ percentage]
                #or None)
removeOutliers=True
normData=True
logTransform=True




#constant parameters:
extractUniqueAA=True # use True when this is the first time to analyze this dataset, otherwise, use False
minNshared=2
onlyProductive=True

mbDataFolder='AllSeqProjects'
folder='%s/SamplesForAnalysis_corrected' %datasetFolder
toSaveName='%s/%s_sampleList' %(datasetFolder, datasetName)
SampleList=gen_sampleList_from_Folder(folder,toSaveName=None)
SampleListName='%s_sampleList' %datasetName
libPrepMethod=None
filterGenotek=True
filterMinimalReads=9000000
filterlibPrepMethod=libPrepMethod
groupFunction='noOutlierMean'
nSTD=5
nMinSamples=3
ignoreNotSameYear=True
removeSamePerson=False




df=genTCRorMBdfWithManipulations(DFtype,genDF,toBinary,removeOutliers,normData,logTransform,
                                 minVal,minSharedT,percShared,
                                 mbLevel,useShortName,datasetFolder,datasetName,extractUniqueAA,
                                minNshared,onlyProductive,mbDataFolder,SampleList,
                                  SampleListName,filterlibPrepMethod,filterGenotek, 
                                  groupFunction, nSTD, nMinSamples, ignoreNotSameYear, removeSamePerson)


## compare phenotypes only and phens+PCA+TCRfeatures using optimized models:

In [ ]:
onlyPhen_list=[True,False]
modelAndParams_list=[(LogisticRegression,'LogisticRegression',{'C':10,'penalty':'l1'}),
                   (lgb.LGBMClassifier,'LGBMClassifier',{'learning_rate': 0.1, 'n_estimators': 1000,'max_depth': 3,'num_threads':2,
             'bagging_fraction':0.5,'bagging_freq':1, 'bagging_seed':3}),
                   (XGBClassifier,'XGBClassifier',{'learning_rate': 0.05, 'n_estimators': 100,'max_depth': 6,'num_threads':2,
             'bagging_fraction':0.5,'bagging_freq':1, 'bagging_seed':3})]



for n,modelAndParams in enumerate(modelAndParams_list):
    for onlyPhen in onlyPhen_list:
        if onlyPhen:
            XName='PNP530Cardio126ss12500Matched83_phenotypesOnly'
            useTCRfeatures=False #True/False
            usePCAdf=False #True/False    
        else:
            XName='PNP530Cardio126ss12500Matched83_phenotypes&PCA&TCRfeatures'
            useTCRfeatures=True #True/False
            usePCAdf=True #True/False

        model=modelAndParams[0]
        modelName=modelAndParams[1]
        model_params=modelAndParams[2]


        print n,modelName

        # (1) general definitions:
        datasetType='PNP_Cardio' #PNP/PNP_Cardio
        #nSeqSS=9000 #12500/9000/112500/'all'

        #(2) model definitions:
        ResultFolder='%s/TCR_real_data/Predictions' %MyPath
        if not isdir(ResultFolder):
            makedirs(ResultFolder)

        n_splits=10
        useCV=True
        stratifiedCV=True

        #(3) target definition:
        targetDF=pd.DataFrame(index=['BD'+str(x) for x in range(1,1100)])
        targetDF['isCardio']=np.where(targetDF.index.str.replace('BD','').astype(int)>949,1,0)
        targetName='isCardio'
        YName=targetName

        #  (4) feature composition:

        ## naming instructions: phenotypes: indicate old or new, TCRdf-indicate threshold to inclusion and binary/RA, 
        ## PCA- indicate n_comp and is Sparse, indicate feature selection
        usePhenotype=True #True/False
        useTCRdf=False #True/False

        # (5) phenotypeDF:

        genPhenotypeDF=True #True/False. no sense for 'True'  if usePhenotype=False
        phenotypefile=None
        phenotypeDFname='newSmall' #('small','allNum and more- see step #6.2


        # (6) TCRdf:
        datasetFolder='%s/TCR_real_data/PNP530Cardio126Combined/MatchedSamples/ss12500rep1' %MyPath
        datasetName='PNP530Cardio126_ss12500rep1_Matched'  
        TCRdfName='sharingMatrix_PNP530Cardio126_ss12500rep1_Matched_minNshared2_RA_onlyProductiveTrue__percShared10_OLtrimmed_binary'


        #(7) PCAdf (based on the TCRdf selected)
        n_comp=100 # int or None 
        isSparse=True #True/False

        #(8) TCRfeatures:
        getTCRfeatures=True #True/False. do not use True when useTCRfeatures=False
        TCRfeatureDF=None
    #     PNPfeatureFile='%s/TCR_real_data/PNP530_SubSampled12500data_rep1/featureSummaryDFs/PNP530_ss12500_rep1_allFeatures' %MyPath
    #     PNPfeatureDF=pd.read_pickle(PNPfeatureFile)
    #     CardiofeatureFile='%s/TCR_real_data/CardioSamples/Cardio126_SubSampled12500data_rep1/featureSummaryDFs/Cardio126_ss12500_rep1_allFeatures' %MyPath
    #     CardiofeatureDF=pd.read_pickle(CardiofeatureFile)

    #     TCRfeatureDF=pd.concat([PNPfeatureDF,CardiofeatureDF])

        #(9) X and y processing:
        #sampleList to filter, and sampleListName:
        mbDataFolder='AllSeqProjects'
        folder='%s/SamplesForAnalysis_corrected' %datasetFolder
        toSaveName='%s_sampleList' %datasetName
        sampleList=gen_sampleList_from_Folder(folder,toSaveName=None)
        sampleListName='%s_sampleList' %datasetName

        filterFeaturesByCorr=None #None or float between 0 and 1
        featureSelectionUsingModel=False # True/False
        C=0.5 #None (if none, C will be set to 0.1) or float between 0 and 1
        filterFeaturesByUnivariate=f_classif     # None/chi2/f_classif/mutual_info_classif
        k=50 #None (will be set to 50 / int - defines number of k best))
        scale=False

        predictionPipeline(datasetType,ResultFolder,model,modelName,model_params,n_splits, useCV,stratifiedCV,
                              XName,usePhenotype,useTCRdf,useTCRfeatures,usePCAdf,
                              YName,targetDF,
                              genPhenotypeDF,phenotypefile,phenotypeDFname,
                              datasetFolder,datasetName,TCRdfName,n_comp,isSparse,
                              getTCRfeatures,TCRfeatureDF,sampleListName,sampleList,
                              filterFeaturesByCorr,featureSelectionUsingModel,scale,C,
                              filterFeaturesByUnivariate,k)



# run again predictions with different models for full cohorts: 210818